# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

## Imports

In [1]:
from tqdm.notebook import trange, tqdm

import glob
import pathlib
import opendatasets as od
import os
from dotenv import load_dotenv

import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU'), tf.__version__

2025-02-09 09:42:44.359373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739094164.371021   77043 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739094164.374323   77043 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 09:42:44.387203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-09 09:42:46.110598: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL

([], '2.18.0')

2025-02-09 09:42:46.110709: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 565.77.0
2025-02-09 09:42:46.110721: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 565.77.0
2025-02-09 09:42:46.110725: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 565.77.0


## Download datasets

In [2]:
# Loading kaggle keys
load_dotenv()

True

In [3]:
%load_ext dotenv
%dotenv

In [4]:
from hydra import initialize, compose
from omegaconf import OmegaConf
from hydra import main
# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


In [5]:

if len(os.listdir(cfg.DATASET.DATASET_DIR)) == 0:
    # Download the dataset
    od.download(dataset_id_or_url=cfg.DATASET.BONE_FRACTURE_DETECTION_DATASET_URL,
                data_dir=cfg.DATASET.DATASET_DIR)

## Loading Images

In [6]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

In [ ]:
TRAIN_DIR = cfg.DATASET_DIRS.TRAIN_DIR
VALIDATION_DIR = cfg.DATASET_DIRS.VALIDATION_DIR
TEST_DIR = cfg.DATASET_DIRS.TEST_DIR

TRAIN_IMAGE_DIR = f'{TRAIN_DIR}/images'
TRAIN_LABELS_DIR = f'{TRAIN_DIR}/labels'

VALID_IMAGE_DIR = f'{VALIDATION_DIR}/images'
VALID_LABELS_DIR = f'{VALIDATION_DIR}/labels'

TEST_IMAGE = f'{TEST_DIR}/images'
TEST_LABELS = f'{TEST_DIR}/labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE

In [8]:
# TODO: filterr images with label and skip if the  label/file is empty

class PrepareDataset:
    def __init__(self, image_dir: str, label_dir: str) -> None:

        self.images_path = [str(f)
                            for f in sorted(pathlib.Path(image_dir).glob('*.jpg'))]
        self.labels_path = [str(f)
                            for f in sorted(pathlib.Path(label_dir).glob('*.txt'))]

        # Ensure that you have same number of images and labels
        if len(self.images_path) != len(self.labels_path):
            raise ValueError(
                "Number of image files and label files do not match.")
        print(
            f'[INOF] Number of images:{len(self.images_path)} and labels:{len(self.labels_path)}')

    # def load_image(self, image_path: str):
    #     """Loads and preprocesses an image."""
    #     img = cv2.imread(image_path)  # Use cv2 for more image format support
    #     if img is None:
    #         raise ValueError(f"Could not read image: {image_path}")
    #     # Convert to RGB (TensorFlow default)
    #     # img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize if needed
    #     # img = img / 255.0  # Normalize pixel values (important!)
    #     return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def get_dataset(self):
        """Loads and parses YOLOv8 labels."""
        labels = []
        bboxes = []
        image_paths = []

        for image_pth, label_path in tqdm(zip(self.images_path, self.labels_path)):
            # print(image_path, label_path)
            # image = self.load_image(image_path=image_path)
            try:
                with open(label_path, 'r') as f:
                    lines = f.readlines()
            except FileNotFoundError:
                continue

            for line in lines:
                # 0 0.458736435546875 0.3806510419921875 0.3614540244140625 0.389472591796875 0.36892872265625 0.48237111328125 0.457112263671875 0.471341630859375 0.458736435546875 0.3806510419921875
                parts = line.strip().split()
                # print(parts)
                if len(parts) >= 5:  # check if the line has enough elements
                    class_id = parts[0]
                    x = parts[1]
                    y = parts[2]
                    width = parts[3]
                    height = parts[4]
                    print(int(class_id), [x, y, width, height])
                    # YOLO format is already normalized.
                    labels.append(class_id)
                    bboxes.append([x, y, width, height])
                    image_paths.append(image_pth)
                else:
                    print(
                        f"Skipping malformed line in {label_path}: {line}")
                    continue

        # return (tf.ragged.constant(image_paths), tf.ragged.constant(labels), tf.ragged.constant(bboxes))
        return image_paths, labels, bboxes

### Training Dataset setup

In [9]:
preparer_train_ds = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR)
image_paths, labels, bboxes = preparer_train_ds.get_dataset()

[INOF] Number of images:3631 and labels:3631


0it [00:00, ?it/s]

0 ['0.35585933923721313', '0.3968750014901161', '0.4525467435362568', '0.3829741733292004']
0 ['0.458736435546875', '0.3806510419921875', '0.3614540244140625', '0.389472591796875']
0 ['0.4779147490234375', '0.3756653818359375', '0.3804500517578125', '0.369160001953125']
0 ['0.37971050146484375', '0.417125080078125', '0.3261278564453125', '0.6149684370117188']
0 ['0.3167693330078125', '0.5993769916992188', '0.37067339990234377', '0.6550453393554687']
0 ['0.3924151103515625', '0.40119055029296874', '0.3118194545898437', '0.5896512465820313']
0 ['0.6960843097732727', '0.7436781623811101', '0.7742012529485355', '0.7178796789872909']
0 ['0.7556493927710843', '0.8306089524904214', '0.7680564590361445', '0.7299196314176245']
0 ['0.7689014861445783', '0.8131365325670498', '0.7771488246987952', '0.7117592283524904']
5 ['0.3388671576976776', '0.2610837454083322', '0.40056988604893773', '0.19880427666687508']
5 ['0.40056988671875', '0.19880427586206895', '0.338867158203125', '0.2610837463054187']

In [10]:

bbox = tf.ragged.constant(bboxes)
classes = tf.ragged.constant(labels)
image_paths = tf.ragged.constant(image_paths)

data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))

In [ ]:
datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
                          label_dir=VALID_LABELS_DIR)
bbox = tf.ragged.constant(bboxes)
classes = tf.ragged.constant(labels)
image_paths = tf.ragged.constant(image_paths)

data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))

In [ ]:
# # Batching, shuffling, prefetching (Essential for training)
# BATCH_SIZE = 32
# dataset = dataset.shuffle(buffer_size=len(image_files))  # Shuffle the dataset
# dataset = dataset.batch(BATCH_SIZE)
# dataset = dataset.prefetch(
#     buffer_size=tf.data.AUTOTUNE)  # Optimize data loading

# print(dataset.take(1))
# for images, labels in dataset.take(1):  # take only one batch
#     print("Image shape:", images.shape)
#     # print("Labels shape:", labels.shape)
#     print("Example labels:", labels.numpy())  # Access the label data